# 관심주제 분류
①보건복지부(서민금융)	http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do?searchIntClId=15&welSrvTypeCd=03	
㉑보건복지부(건강)	http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do	
㉔보건복지부(주거)	http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do	
㉛보건복지부(고용)	http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do	
㊳보건복지부(교육)	http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do	
㊵보건복지부(문화)	http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do

----------------------------------------------------------------------------------

㊳보건복지부(교육)	http://online.bokjiro.go.kr/apl/faq/aplFaq.do	
㉛보건복지부(고용)	http://online.bokjiro.go.kr/apl/faq/aplFaq.do	


1. '관심주제' - 교육, 고용, 주거, 건강, 서민금융, 문화 中 택일
2. Q&A 리스트 클릭
3. Q&A 내용 크롤링
4. 페이지 이동

In [1]:
import time
import os
import json
from datetime import datetime

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm_notebook

## 1. 관심주제 택1
http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do?searchIntClId=<숫자>  
11: 교육, 12: 고용, 13: 주거, 14: 건강, 15: 서민금융, 16: 문화


## 2. 상세 페이지 url 수집

In [11]:
def get_detailUrl(url):
    """
    (string) -> list
        Args:
            url(string): Q&A 리스트
    Return:
        url_list: detail page link
    """
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    # url(자바스크립트) 주소
    url_list = soup.select("dl > dt > a")
    
    # 자바스크립트 주소에서 필요한 부분만 가져오기 위한 전처리
    temp_coord = [url['href'][18:] for url in url_list]
    coord = [[temp.replace("(", "").replace(")","").replace("'","").split(',')[0], temp.replace("(", "").replace(")","").replace("'","").split(',')[1]] for temp in temp_coord]
    
    url_list = list()
    for temp in coord:
        url = 'http://www.bokjiro.go.kr/welInfo/retrieveGvmtWelInfo.do?searchIntClId=11&searchCtgId=%s&welInfSno=%s'%(temp[0], temp[1])
        url_list.append(url)
    return url_list

In [15]:
def get_detailCnt(url):
    """
    (string) -> list
        Args:
            url(string): detail_page url
        Return:
            detail_list: ['질문', '간단 설명', '누가', '어떤', '어떻게', '연락처', '사례로 알아보기']
    """
    
    options = webdriver.ChromeOptions()	
    options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)

    # Q&A 정보를 담은 리스트: info_list
    # ['제목', '간단 설명', '지원 대상', '지원 내용', '지원 방법', '추가 문의', '사례 설명']
    info_list = list()
    title = driver.find_elements_by_tag_name('#contents > div.bokjiDetailWrap > h4 > span')[0].text
    simple = driver.find_elements_by_tag_name('#contents > div.bokjiDetailWrap > div')[0].text
    info_list.append(title)
    info_list.append(simple)

    info_box = driver.find_elements_by_tag_name('div.bokjiServiceView')
    # info_box의 길이가 4개 (지원 대상, 지원 내용, 지원 방법, 추가 문의)가 정석임 -> 3개인 경우는 print(url)로 별도 표시
    if len(info_box) == 4:
        for box in info_box:
            temp_box = box.text.replace('\n','').replace('\t','').replace('\r','')
            info_list.append(temp_box)
        try:
            example = driver.find_elements_by_tag_name('#instanceView > div > div > img')[0].get_attribute('src')
        except:
            example = "NULL"
        info_list.append(example)
    else:
        print('특이 케이스: ', url)

    driver.close()
    
    return info_list

In [16]:
def main():
    # 카테고리별 넘버링
    subject = ['11', '12', '13', '14', '15', '16']
    
    # 11: 교육, 12: 고용, 13: 주거, 14: 건강, 15: 서민금융, 16: 문화
    for s in subject:
        if s == '11':
            cate = '교육'
        elif s == '12':
            cate = '고용'
        elif s == '13':
            cate = '주거'
        elif s == '14':
            cate = '건강'
        elif s == '15':
            cate = '서민금융'
        else:
            cate = '문화'
        fileName = '복지로_한눈에_보는_복지정보_%s'%(cate)
        
        csv_file = open(str(fileName) + '.csv', "w", encoding = 'utf-8-sig', newline = '')
        wr = csv.writer(csv_file)
        wr.writerow(['질문', '간단한 설명', '혜택 대상', '혜택 내용', '신청 방법', '고객센터', '사례로 알아보기'])
        
        for page in range(1,11):
            url = 'http://www.bokjiro.go.kr/welInfo/retrieveWelInfoBoxList.do?searchIntClId=%s&pageIndex=%s'%(s, page)

            # detail url 얻기
            try: 
                url_list = get_detailUrl(url)
                if len(url_list) == 0:
                    print('페이지 이동 끝')
                    break
            except:
                print('페이지 이동 끝')
                break

            # detail 정보 수집
            for url in url_list:
                detail_list = get_detailCnt(url)
                print(detail_list)
                wr.writerow(detail_list)
        print('============================')
        print(fileName, '저장완료')
        print('============================')
        csv_file.close()


In [17]:
if __name__ == "__main__":
    main()

['아동통합서비스지원(드림스타트사업)', '취약계층 아동에게 맞춤형 통합서비스를 제공하여\n건강한 성장과 발달을 도모하고 공평한 출발 기회를 보장함으로써\n건강하고 행복한 사회구성원으로 성장할 수 있도록 지원합니다.', '누가 받을 수 있나요?지원대상0세(임산부)이상 만 12세(초등학생 이하) 아동 및 가족이 대상입니다.만 13세 이상의 초등학교 재학 아동 포함수급자 및 차상위계층 가정, 보호대상한부모가정(조손가정 포함), 학대 및 성폭력피해아동 등에 대한 우선지원 원칙선정기준사업지역에 거주하는 해당 연령 아동과 그 가족, 임산부(0세)를 대상으로 아동 양육환경 및 발달 상태에 대한 사정을 통해 개입이 필요한 아동을 서비스 대상 아동으로 선정합니다.만 13세 이상 아동 중 초등학교 재학 아동 포함, 수급자 및 차상위 계층 가정, 보호대상 한부모 가정(조손가정 포함), 학대 및 성폭행 피해 아동 등에 대한 우선지원 원칙※ 아동에 대해 정기적인 사정을 실시하여 대상자 재선정', '어떤 혜택을 받을 수 있나요?지원내용가정 방문을 통해 서비스 대상 아동을 발굴하여 통합적으로 사례를 관리하는 기본 서비스를 지원합니다.아동발달 영역별(신체/건강, 인지/언어, 정서/행동, 부모의 양육) 목표를 달성하기 위해 모든 드림스타트 사례관리 대상자에게 아래와 같은 필수 서비스를 제공합니다.아동: 건강검진(성장발달 스크리닝 포함), 예방접종, 영양 교육, 응급처치 교육, 아동권리 교육, 인터넷 중독 예방 교육, 소방 및 안전 교육, 학대 및 (성)폭력 예방 교육(총 8종)임산부: 산전 및 산후 검진, 예비부모 교육(총 2종)부모: 자녀발달 및 양육 교육(총 1종)시정 결과에 따라 대상자별로 지원하는 맞춤형 서비스를 지원합니다.물품 지원 및 후원자 연계 등의 기타 서비스를 지원합니다.', '어떻게 신청하나요?신청방법시/군/구청에 방문하거나 전화 또는 우편 및 인터넷 등으로 신청합니다.지원절차다음과 같은 순서로 지원합니다.초기상담 및서비스 신청시/군/구청에서 초기상담을 받고 서비스 신청

['(북한이탈주민) 탈북청소년 교육지원', '북한에서 남한으로 이주한 청소년이 학교생활에 적응하여\n우리 사회의 건강한 구성원이 될 수 있도록 돕습니다.', '누가 받을 수 있나요?지원대상탈북청소년(제3국 출생 탈북민 자녀 포함)을 지원합니다.', '어떤 혜택을 받을 수 있나요?지원내용일반정규학교 적응에 어려움을 겪는 탈북청소년들을 위해 운영중인 탈북청소년학교(특성화학교 및 인가 대안학교)와 미인가 및 민간 대안교육시설을 지원합니다.탈북 청소년의 방과후 학습지도를 위한 방과후 공부방을 지원합니다.일반학교에 재학중인 탈북학생의 학교 적응 및 맞춤형 교육을 지원하기 위해 탈북학생 통일전담교육사를 운영합니다.탈북 중/고교생 및 대학생 대상으로 장학금 및 제3국 출생 정원내 특례 입학을 지원하기 위한 탈북청소년 장학사업을 실시합니다.WEST(한미취업연수) 및 호주어학연수를 지원합니다.9세 이상 탈북청소년 및 제3국 출생 탈북민 자녀 대상 인터넷을 이용하여 영어문법과 발음, 어휘 등을 배울 수 있도록 실시간 화상영어교육강좌의 수강을 지원합니다.만 3세~초등학생이 주 1회 방문 학습을 받을 수 있도록 방문 학습지 비용을 지원합니다.대학 진학을 준비하는 탈북 청소년과 학부모에게 대학입시박람회를 통해 입시정보를 제공합니다.대학 입학을 앞둔 탈북 청소년 및 탈북 대학생에게 온라인 강의 및 오프라인 교육, 멘토링을 실시하는 예비대학 과정을 제공합니다.', '어떻게 신청하나요?신청방법남북하나재단을 방문하거나 우편, 전화를 이용하여 신청합니다.지원절차다음과 같은 순서로 지원합니다.초기상담 및서비스 신청북한이탈주민 지원재단에서 초기상담을 받고 서비스 신청사실조사 및 심사북한이탈주민 지원재단에서 사실을 조사하고 심사를 진행서비스 결정북한이탈주민 지원재단에서 서비스를 결정서비스 제공북한이탈주민 지원재단에서 서비스를 제공', '아직 궁금한 것이 있어요!문의처남북하나재단 콜센터 ☎ 1577-6635관련 사이트남북하나재단 http://www.koreahana.or.kr근거법령북한이탈주민의 보

['장기복무 제대군인 취업지원', '오랫동안 복무하고 전역한 제대군인이 원활하게 사회에 복귀할 수 있도록\n취업을 지원하고, 특기와 적성에 맞는 직업교육 훈련을 제공함으로써\n취업역량을 강화하고 취업률을 높입니다.', '누가 받을 수 있나요?지원대상군에서 10년 이상 복무 후 전역한 장교, 준사관, 부사관에게 취업을 지원합니다.관할 제대군인지원센터 회원으로, 전역일 기준 5년 이상 복무자로서 1년 이내 전역 예정이거나 교육시작일 및 취업역량비 신청일 현재 전직교육기간 중인 자에게 취업역량 교육비를 지원합니다.주소지 관할 보훈(지)청에 중장기복무 제대군인 지원대상으로 결정(등록)된 자로서, 전역 후 3년이 지나지 않은 미취업 상태의 제대군인에게 직업교육훈련 바우처를 지원합니다.선정기준취업지원의 선정기준은 다음과 같습니다.군에서 10년 이상 복무 후 전역한 장교, 준사관, 부사관으로서 취업희망 신청서를 제출한 날 현재 전역 후 3년이 경과되지 아니한 자전역 후 3년이 경과된 경우, 생활수준조사 결과 기준금액 이하인 자', '어떤 혜택을 받을 수 있나요?지원내용아래는 취업 지원의 세부사항입니다.국가기관, 지방자치단체, 일상적으로 20인 이상(제조업체의 경우에는 200인 이상)을 고용하는 공기업체와 사기업체 및 단체에 취업을 지원합니다.취업 지원은 1회입니다(업체 등의 폐업, 통합, 합병 기타 본인의 책임이 없는 사유로 퇴직한 경우에는 재취업 지원).채용 시, 우대사항입니다.- 제대군인(의무복무자 포함)- 각종 채용시험에 응시 가능한 상한연령을 군복무기간에 따라 1～3세 연장- 채용시험 결과가 동점인 경우에 제대군인을 우선적으로 합격시킴- 호봉과 임금을 결정할 때, 군 복무 경력을 포함시킴', '어떻게 신청하나요?신청 방법취업지원은 보훈(지)청을 방문하거나 우편, 온라인(http://www.vnet.go.kr)으로 신청합니다.지원 절차다음과 같은 순서로 지원합니다.초기상담 및서비스 신청취업지원은 보훈(지)청에서 초기상담을 하고, 서비스 신청사실조사 및 심사취업지원은 

['국가보훈대상자 학습보조비 지급', '학용품을 구입하는 등의 교육에 필요한 부수적인 교육비를 지원합니다.', '누가 받을 수 있나요?지원대상중학교, 고등학교에 재학 중인 유공자 본인 및 그 자녀, 전몰·순직·사망·행방불명자의 배우자를 지원합니다.대학교에 재학 중인 국가유공자(애국지사, 고엽제후유의증환자, 5.18 민주유공자, 특수임무유공자, 보훈보상대상자 포함) 본인 및 전몰·순직·사망·행방불명자의 배우자를 지원합니다.특수교육 대상자로서 특수교육을 받는 교육지원대상자에게 지원합니다.', '어떤 혜택을 받을 수 있나요?지원내용교육지원대상자의 과정별 최대 학습보조비 지급기간은 다음과 같습니다.중학교(과정): 6학기(이내)고등학교(과정): 6학기(이내)2년제 대학(과정): 4학기(이내)3년제 대학(과정): 6학기(이내)4년제 대학(과정): 8학기(이내)5년제 대학(과정): 10학기(이내)6년제 대학(과정): 12학기(이내)학용품 등을 구입할 수 있도록 다음의 금액을 지급합니다.중학생: 12만4,000원인문계 고등학생: 14만4,000원전문계 고등학생: 18만6,000원대학생: 23만6,000원특수학교 유·초등부: 53만4,000원특수학교 중·고등부: 71만8,000원', '어떻게 신청하나요?신청방법관할 보훈청에 신청합니다.☞ 국가보훈 지(방)청 소개지원절차다음과 같은 순서로 지원합니다.보훈대상 등록신청보훈(지)청에서 보훈대상 등록을 신청보훈심사•결정보훈심사위원회에서 보훈대상을 심사하고 결정보훈급여 신청주소지 관할 보훈(지)청에서 보훈급여를 신청지원대상여부 확인보훈(지)청에서 지원대상 여부를 확인보훈급여 지급보훈(지)청에서 대상이 되면 보훈급여를 지급', '아직 궁금한 것이 있어요!문의처국가보훈처 보훈상담센터 ☎ 1577-0606관련 사이트국가보훈처 http://www.mpva.go.kr/서식/자료2020년 교육지원업무처리규정근거법령보훈보상대상자 지원에 관한 법률국가유공자 등 예우 및 지원에 관한 법률5·18민주유공자예우에 관한 법률특수임무유공자 예우 및 단체설립에

['직장어린이집 설치 지원', '근로자의 육아 부담을 덜고 여성의 경제활동 참여를 촉진하기 위하여\n직장어린이집의 설치 및 운영을 지원합니다.', '누가 받을 수 있나요?지원대상직장 어린이집을 단독 또는 공동으로 설치, 운영하는 고용보험 가입 사업주나 사업주 단체에게 지원합니다.', '어떤 혜택을 받을 수 있나요?지원내용직장어린이집 시설 설치비와 교재·교유비를를 지원합니다.시설설치비는 시설전환 소요비용의 60%~90%를 지원합니다.· 대규모기업 : 단독형(3억), 공동형(6억)· 우선지원대상기업 : 단독형(4억), 공동(10억~20억)* 단, 시설매입비는 우선지원대상기업만 40% 한도에서 지원교재·교구비는 소요비용의 60%~90%를 지원합니다.신규 : 대규모기업(5천만원), 우선지원대상기업(7천만원)교체비 : 3천만원보육교사 및 보육시설의 장, 취사부 1인당 월 60만원(중소기업 월 120만원) 한도로 월평균 근무시간에 따라 인건비를 차등 지원합니다.중소기업에서 운영하는 직장어린이집에 대해 월 200~520만원 한도로 보육아동의 현원 규모에 따라 운영비를 차등 지원합니다.직장어린이집 설치 및 여성고용친화시설 융자를 지원합니다.시설건립비, 매입비 등 최고 7억원(공동 9억원) 한도로 연 2.0% 이율로 융자(우선지원대상기업은 연 1.0%), 상환조건은 3년 거치 5년 균등분활상환', '어떻게 신청하나요?신청방법사업주가 근로복지공단 직장보육지원센터에 방문하거나 우편으로 신청합니다.지원절차다음과 같은 순서로 지원합니다.서비스 제공근로복지공단 직장보육지원센터에서 직장어린이집 설치 및 교재교구지원, 고용노동부 고용센터에서 보육교사 등의 인건비와 중소기업 운영비 지원', '아직 궁금한 것이 있어요!문의처근로복지공단 직장보육지원센터 ☎ 02-2670-0410~9근거법령고용보험법고용보험법 시행규칙고용보험법 시행령', 'http://www.bokjiro.go.kr/nwel/img/serviceContents/story/S241.jpg']
['시간제보육', '가정 양육 시에도 

...

['치매 치료관리비 지원사업', '노후에도 안정적인 삶을 누릴 수 있도록 치매를 조기에 치료하고 지속적으로 관리하여 증상을 호전시키거나 심화를 예방합니다.', '누가 받을 수 있나요?지원대상해당 지역 주민(주민등록기준) 중 보건소(치매상담센터)에 치매환자로 등록된 자가 선정기준을 만족하면 지원합니다.선정기준다음의 기준을 모두 충족하여야 합니다.연령기준: 만 60세 이상인 자(초로기 치매환자도 예외적으로 선정 가능하나, 이 경우에도 진단기준과 치료기준, 소득기준은 반드시 충족해야 함)진단기준: 의료기관에서 상병코드 F00~F03, G30 중 하나 이상으로 진단을 받고, 반드시 보건소(치매상담센터)에 치매환자로 등록되어 있어야 함치료기준: 처방전 사본 또는 영수증에 치매치료약이 기재되어 있고, 치매치료약 복용 여부가 확인되어야 함소득기준: 기준 중위소득의 120% 이하인 경우, 의료급여 수급자는 소득기준을 충족하는 것으로 판정하여 지원하고, 국민건강보험 가입자 중 건강보험료 본인부담액이 치매치료관리비 지원대상자 소득판정기준 이하인 분을 지원함보훈대상자 의료지원 대상자에게는 지원하지 않습니다.', '어떤 혜택을 받을 수 있나요?지원내용치매치료관리비 보험급여분 중 본인부담금(치매약제비 본인부담금 + 약 처방 당일의 진료비 본인부담금)을 월 3만원(연 36만원) 이내에서 실비로 지원합니다.치료제는 복용 개월수에 따라 일괄 지급합니다.', '어떻게 신청하나요?신청방법시/군/구 보건소를 방문하거나 우편으로 신청합니다.지원절차다음과 같은 순서로 지원합니다.지원 신청보건소에서 상담하고 지원을 신청대상자 적격여부판단 및 결정보건소에서 대상자 적격 여부를 판단하고 결정대상자 명단 통보보건소에서 대상자 명단을 통보본인부담금 환급국민건강보험공단에서 본인부담금을 환급', '아직 궁금한 것이 있어요!문의처보건복지상담센터 ☎ 129치매상담콜센터 ☎ 1899-9988관련 사이트보건복지상담센터 http://www.129.go.kr치매상담콜센터 http://www.nid.or.kr서식/자료20

['의료급여 틀니·치과임플란트', '노인 수급권자의 경제적 부담을 완화하고, 치아건강 증진을 위해 틀니 및\n치과 임플란트에 대하여 의료급여를 지원합니다.', "누가 받을 수 있나요?지원대상만 65세 이상의 의료급여수급권자이면서, 치과 병·의원에서 발급받은 틀니(치과 임플란트) 등록신청서를 관할 시·군·구청 또는 읍·면·동에 방문하여 제출·등록하신 분을 지원합니다.(사전등록제 실시)틀니는 동일부위(상악·하악) 동일종류(완전틀니·부분틀니)의 경우 7년에 1회 급여적용이 원칙이며, 구강상태가 심각하게 변화되어 새로운 틀니가 필요한 경우 7년 이내에 재 제작 가능치과 임플란트는 1인당 평생 2개 급여 적용진료전달체계: 의료급여 진료절차(1차→2차→3차) 준수선정기준틀니에 대한 기준은 다음과 같습니다.급여시작일틀니 급여시작일급여내용 시작일자레진상 완전틀니 2012년 7월 1일부터~금속상 완전틀니 2015년 7월 1일부터~금속상 부분틀니 2013년 7월 1일부터~사후 유지관리 2012년 10월 1일부터~대상자: 만 65세 이상 의료급여 수급권자* (12년 7월) 만 75세 이상→(15년 7월) 만 70세 이상→(16년 7월)만 65세 이상급여대상: 레진상 완전틀니, 금속상 완전틀니, 클라스프 부분틀니, 사전 임시 틀니, 사후 유지관리* 사후 유지관리는 틀니 급여화 이전에 비급여로 제작하여 사용하고 있는 수급권자를 포함하여 급여 지원본인부담: 1종 수급권자 5%, 2종 수급권자 15%, 부분틀니 지대치는 별도 본인부담(비급여)(* '17년 11월부터 1종 수급권자 5%, 2종 수급권자 15% 적용)급여횟수: 원칙적으로 동일부위(상악·하악) 동일종류(완전틀니·부분틀니)의 경우 7년에 1회 급여적용하나, 구강상태가 심각하게 변화되어 새로운 틀니가 필요한 경우 7년 이내에 재 제작 가능하며, 사전등록제 실시로 틀니 수급권자 이력 관리가 가능하므로 중복급여 여부 확인 가능치과 임플란트에 대한 기준은 다음과 같습니다.급여시작일: 2014년 7월 1일부터~대상자: 만 65세 이상 

['매체활용능력 증진 및 역기능 해소\n(청소년 인터넷·스마트폰 중독 치료비 지원)', '인터넷과 스마트폰에 중독된 청소년을 치료하고\n중독의 위기에 놓인 청소년이 중독되는 것을 예방합니다.', '누가 받을 수 있나요?지원대상인터넷·스마트폰 이용습관 진단 전수조사 결과 고위험군으로 진단 받은 만 19세 미만 청소년이 지원대상입니다.기준 중위소득 60% 이하, 한부모가정, 소년소녀가장 등 취약계층 가정의 청소년을 우선 지원합니다.선정기준취약계층의 경우, CYS-Net 실행위원회 또는 사례판정위원회에서 지원대상을 결정합니다.', '어떤 혜택을 받을 수 있나요?지원내용인터넷·스마트폰 중독 예방 및 상담·치료를 지원합니다.개별상담이나 집단상담, 전화상담, 사이버상담 등 지원기숙형 치유캠프, 가족치유캠프, 가족관계 증진 프로그램 운영인터넷(게임) 중독 예방 및 해소 프로그램 개발인터넷·스마트폰 과의존 상담전문인력을 양성하고, 전문교재를 개발할 수 있도록 지원합니다.인터넷ㆍ스마트폰 중독 고위험군 청소년의 치료비를 지원합니다.일반 청소년은 최대 30만원, 저소득층이나 취약계층의 청소년은 최대 50만원 지원', '어떻게 신청하나요?신청방법17개 시/도 청소년상담지원센터에 신청합니다.지원절차다음과 같은 순서로 지원합니다.전수조사 실시청소년상담지원센터에서 인터넷 중독에 해당하는 청소년을 조사상담치료대상자명단 송부청소년상담지원센터에서 상담치료 대상자 명단을 송부서비스 결정청소년상담지원센터에서 인터넷 중독 관련 서비스를 제공', '아직 궁금한 것이 있어요!문의처청소년전화 ☎ 1388관련 사이트여성가족부 http://www.mogef.go.kr청소년사이버상담센터 http://www.cyber1388.kr서식/자료2020년 청소년사업 안내근거법령청소년보호법', 'http://www.bokjiro.go.kr/nwel/img/serviceContents/story/S158.jpg']
['특수교육대상자 치료지원서비스', '특수교육대상자의 효과적인 교육을 위해 다양한 서비스를 제공합니다.',

['농촌주택 개량자금 지원', '노후·불량 농어촌 주택 개량 및 신규 주택건축 수요에 대응하여 농어촌 주거환경 개선을 통해 주거복지 실현 및 농어촌 활성화를 도모합니다.', '누가 받을 수 있나요?지원대상농어촌지역(준농어촌지역을 포함하며, 이하 “농촌지역”이라 한다.)에서 본인 소유의 노후·불량주택을 개량하고자 하는 자로서 세대주 또는 배우자*인 자에게 지원합니다.* 배우자는 가족관계증명서상의 배우자를 말하며, 세대주 또는 배우자 중 1명만 신청 가능(이하 동일 적용)다만, 신축의 경우 융자금 대출일 이전에 기존주택을 반드시 철거한 후 건축물대장 및 건축물등기부등본을 말소하여야 하며, 무허가주택 또는 불법건축물 거주자는 기존 주택 철거 사실을 사진 등으로 증명해야 함개량 하려는 주택 이외에 다른 주택을 소유한 경우(세대원* 포함)는 사업 신청 불가* 세대원은 주민등록등본 상에 기재된 세대원을 말하며, 주민등록등본상에 등재되어 있지 않지만 가족관계증명서상의 배우자와 그 세대원도 포함(이하 동일 적용)농촌지역에서 거주하고 있는 주민 중 무주택자(세대원 포함)로서 세대주 또는 배우자인 자에게도 지원합니다.도시지역(洞)에서 농촌지역으로 이주하려는 자로서, 주택 융자대출 신청일 이전까지 도시지역의 주택을 처분*하여 관련 증빙서류 제출이 가능하고, 융자대출 신청일 전까지 「주민등록법」에 따라 사업대상 주택으로 전입신고를 완료할 수 있는 세대주 또는 배우자도 지원 가능합니다.* 융자대출을 받지 않는 경우는 건축물 소유권 등기를 하기 전 까지 도시지역 주택을 처분하지 않거나 소유권 등기를 하기 전 까지 사업대상 주택으로 전입하지 않은 경우 사업대상에서 제외도시지역에 거주하면서 2주택 이상을 소유한 경우(세대원 포함)는 사업 신청 불가※ 「농어촌정비법」에 따른 국고보조금을 지원받는 신규(전원)마을조성사업 입주예정자가 분양받은 조성용지에 주택을 건축하는 경우 위의 규정은 미적용건축행위 진행 중인 경우의 사업신청의 경우 사업대상자 요건을 충족하는 자가 건축법에 따른 건축허가를 받

['일반상환학자금 대출', '개인 사정에 따라서 거치기간과 상환기간을 합리적으로 조절하여\n상환할 수 있는 학자금 대출제도를 운영합니다.', '누가 받을 수 있나요?지원대상국내의 대학(대학원 포함)에 다니고 있거나 입학할 예정인 대한민국 국민을 대상으로 지원합니다.선정기준학자금 지원 구간에 관계없이 학부생 및 대학원생은 일반 상환 학자금대출 이용이 가능합니다.대출 신청일 현재 만 55세 이하*인 사람을 지원합니다.* 단, 만 55세 이전에 대학(원)에 입학하여 중단 없이 학업을 지속하는 경우 만 59세까지 일반 상환 학자금대출 가능(만 55세에 대출을 신청하고 만 56세에 해당 학기에 입학하여 학업을 지속하는 자 포함)하나, 해당 학위과정을 마친 이후에는 대출 불가직전학기 성적 70/100점(C학점) 이상이어야 합니다.* 단, 신입생군, 장애인의 경우 성적 기준 적용 제외직전학기 소속 대학 최소이수학점 또는 12학점 이상 이수하여야 합니다.* 단, 신입생군, 장애인, 대학원생, 졸업학년 학부생인 경우 이수학점 기준 적용 제외지원대상은 한국장학재단이 정하는 기본적 신용요건*을 충족하며, 대출제한 대상자가 아니어야 합니다.* 재단 학자금 대출 연체 및 부실채권 보유자, 한국신용정보원 신용도 판단정보(신용유의자), 공공정보 보유자(개인회생, 파산 등) 등은 대출 제한', '어떤 혜택을 받을 수 있나요?지원내용등록금 소요액 전액(한도 있음)을 대출할 수 있습니다.※ 대출 금액 총 한도대학(전문대포함): 4천만원5·6년제 대학 및 일반·특수대학원: 6천만원의·치·한의계열 대학(원) 및 전문대학원: 9천만원생활비는 학기당 최대 150만원(생활비 한도 내에서 4회까지 분할대출 허용)까지 대출 가능합니다.대출금리는 2.0%(고정금리, 2020년도 1학기)입니다.대출 기간은 최장 20년(거치기간 10년 + 상환기간 10년) 이내에서 선택합니다.', '어떻게 신청하나요?신청방법한국장학재단 홈페이지(www.kosaf.go.kr)에서 서비스를 신청합니다.지원절차다음과 같이 진행합니